Author: Stephen C. Wein, Quantum Information Research Scientist at Quandela
Email:  stephen.wein@quandela.com
Date:   September 20th, 2023

Dependencies:
numpy
qutip
perceval-quandela
itertools
tensorflow

Description:
The first part of the notebook simulates scattering probabilities for a two-level emitter driven by a square pulse in the semi-classical approximation using both the standard recursive integration approach and the numerical ZPG method. It is used to generate results presented in Figure 2 of the paper "Exponential speedup for simulating photon counting from dynamic quantum sources" available online:
 https://arxiv.org/abs/2307.16591.

The second part of the notebook extends the ZPG method to multi-source/multi-mode simulations of the same emitter type presented in Figure 3 of the same paper. This is done to validate numerical results against ideal discrete-variable simulations provided by Perceval, and to give an example of the computational time scaling.

In [3]:
import numpy as np
import qutip as qt

## Part 1: single mode statistics

System setup

In [4]:
Hd = (qt.destroy(2) + qt.create(2))/2  # driving Hamiltonian (sans Omega(t))
istate = qt.fock(2, 0)  # initial state = |g>
area = 10 * np.pi
width = 2
Ht = [Hd, lambda t, args: area/width if 0 < t < width else 0]

In [3]:
# See Mathematica notebook exact_values.nb
exact = [0.36766965348784736,
         0.09251845596353651,
         0.39255037245458413,
         0.09567822259508901,
         0.04140436181229831,
         0.008321043393002221,
         0.001591061140004126]

In [4]:
def r_error(p, q):
    return abs(p-q)/(p+q)

a) Recursive integration using qutip.scattering

In [5]:
def pn_scattering(n, tlist):
    return qt.scattering_probability([Ht], istate, n_emissions=n, c_ops=[qt.destroy(2)], tlist=tlist)

In [6]:
# 80 points chosen to get p(3) error near 0.01, tlist could be optimised a bit more but won't change scaling
tlist = np.linspace(0, 12, 80)
pnset_scattering = [pn_scattering(n, tlist) for n in range(0, 4)]

for i in range(0, 4):
    print(f"p({i}) =", pnset_scattering[i], "  \t", "error = ", r_error(exact[i], pnset_scattering[i]))

p(0) = 0.36765651424511825   	 error =  1.7868591253346313e-05
p(1) = 0.08962203553205443   	 error =  0.015902122629070627
p(2) = 0.39661183319409266   	 error =  0.005146547453029737
p(3) = 0.09379035803326696   	 error =  0.009963998017830251


b) ZPG + FFT method

In [7]:
def p0_zpg(eta, tlist):
    options = qt.Options(atol=10**-16, rtol=10**-16, nsteps=10000)
    J = -eta * qt.sprepost(qt.destroy(2), qt.create(2))
    return np.trace(qt.mesolve([Ht], istate, tlist=tlist, c_ops=[qt.destroy(2), J], options=options).states[-1].full())

def pn_zpg(N, tlist):
    v_configs = [1 - np.exp(-1.j * 2 * np.pi * n / (N + 1)) for n in range(0, N + 1)]
    zpg_points = [p0_zpg(eta, tlist) for eta in v_configs]
    return list(abs(np.fft.ifft(zpg_points)))

In [8]:
pnset_zpg = pn_zpg(N=14, tlist=[0, 1, 30]) # give a point in the middle of the pulse to help mesolve

for i in range(0, 7):
    print(f"p({i}) = ", pnset_zpg[i], " \t", "error = ", r_error(exact[i], pnset_zpg[i]))

p(0) =  0.3676696534878733  	 error =  3.52540757430955e-14
p(1) =  0.0925184559637296  	 error =  1.0434764596915593e-12
p(2) =  0.39255037245443986  	 error =  1.8376429137990085e-13
p(3) =  0.09567822259506605  	 error =  1.2002594842890912e-13
p(4) =  0.04140436181227855  	 error =  2.3864599010028235e-13
p(5) =  0.008321043393000896  	 error =  7.963717247051931e-14
p(6) =  0.0015910611400009526  	 error =  9.972777535312083e-13


Note that, mesolve is not the fastest way to solve this particular system. Because it is time-independent and the ZPG is sparse, two applications of a Krylov subspace expmv algorithm could solve it in about 1ms per ZPG sampling point. But, since qt.scattering uses mesolve, I use it as well to solve the ZPG for an apples-to-apples comparison.

## Part 2: Multimode statistics using the ZPG method

In [9]:
import perceval as pcvl
import string
import time
from itertools import product
from tensorflow import constant, einsum, complex64 # could probably be replaced with numpy

In [10]:
# Hamiltonian for 'size' identical two-level emitters
def square_drive(size):
    Hd = (qt.destroy(2) + qt.create(2))/2
    Htotal = sum(qt.tensor([qt.identity(2)]*n + [Hd] + [qt.identity(2)]*(size - n - 1)) for n in range(0, size))
    return [Htotal, lambda t, args: args['area']/args['width'] if 0 < t < args['width'] else 0]

# list of lowering operators
def sigma_vector(size):
    return [qt.tensor([qt.identity(2)]*n + [qt.destroy(2)] + [qt.identity(2)]*(size - n - 1)) for n in range(0, size)]

# Constructs vector of detector jump operators
def J_vector(U, size):
    sigmas = sigma_vector(size)
    return [sum(U[k, i] * U.dag()[j, k] * qt.sprepost(sigmas[i], sigmas[j].dag())
                for i in range(0, size) for j in range(0, size)) for k in range(0, size)]

# Simulates the zero-photon probability from the ZPG
def p0_zpg(eta_vec, H, istate,  c_ops, J_vec, tlist, args: dict = None):
    options = qt.Options(atol=10**-8, rtol=10**-8, nsteps=10000)
    J = -sum(eta_vec[j] * J_vec[j] for j in range(0, len(eta_vec)))
    return np.trace(qt.mesolve(H, istate, tlist=tlist, c_ops=c_ops + [J], options=options, args=args).states[-1].full())

# Threshold detection inverse Z-transform implemented by tensor contraction (small overhead vs FFT)
def td_inverse(tensor):
    rank = len(tensor.shape)
    mat = constant([[1., 0.], [-1., 1.]], dtype=complex64)
    alphabet = list(string.ascii_lowercase)
    mat_indicies = alphabet[0:rank]
    tensor_indices = alphabet[rank:rank + rank]
    axes_indices = [tensor_indices[i] for i in range(0, rank)]
    mat_indicies = [''.join([mat_indicies[i], axes_indices[i]]) for i in range(0, rank)]
    mat_indicies = ','.join(mat_indicies)
    tensor_indices = ''.join(tensor_indices)
    return np.array(einsum(','.join([mat_indicies, tensor_indices]), *([mat] * rank + [tensor])))

# Computes the probability distribution for unitary U between times tlist[0] and tlist[-1] using
# method = 'PNR' or 'Threshold'. For PNR, each detector contributes 'truncation' number of ZPG points for the FFT.
def pn_zpg(U, tlist, method='PNR', truncation=None, args: dict = None):
    args = {'area': np.pi, 'width': 0.0001} if args is None else args
    size = U.shape[0]
    truncation = size if truncation is None else truncation

    H = square_drive(size)
    J_vec = J_vector(U, size)
    c_ops = sigma_vector(size)

    istate = qt.tensor([qt.fock(2, 0)]*size) # ground state for all emitters

    if method == 'Threshold':
        v_configs = list(product(*[[1, 0]]*size))
        zpg_points = [p0_zpg(list(eta_vec), H, istate, c_ops, J_vec, tlist, args) for eta_vec in v_configs]
        zpg_points = np.reshape(zpg_points, [2]*size)
        return abs(td_inverse(zpg_points))
    elif method == 'PNR':
        v_configs = [1 - np.exp(-1.j * 2 * np.pi * n / (truncation + 1)) for n in range(0, truncation + 1)]
        v_configs = list(product(*[v_configs]*size))
        zpg_points = [p0_zpg(list(eta_vec), H, istate, c_ops, J_vec, tlist, args) for eta_vec in v_configs]
        zpg_points = np.reshape(zpg_points, [truncation + 1]*size)
        return abs(np.fft.ifftn(zpg_points))
    else:
        assert False, "Method doesn't exist."

# Computes the TVD relative to the exact distribution simulated using Perceval with ideal single-photon inputs
def tvd(U, tlist, method = 'PNR', truncation=None, args: dict = None):
    assert any(method == i for i in ['PNR', 'Threshold']), "Method doesn't exist"
    size = U.shape[0]

    t0 = time.time()
    zpg_dist = pn_zpg(U, tlist, method=method, truncation=truncation, args=args)
    tf = time.time() - t0

    p = pcvl.Processor('SLOS')
    p.add(0, pcvl.Unitary(pcvl.Matrix(U.full())))
    p.with_input(pcvl.BasicState([1]*size))
    pcvl_dist = p.probs()['results']

    # Computes the threshold detection distribution from the exact PNR distribution
    pcvl_td = {}
    for k, v in pcvl_dist.items():
        try:
            pcvl_td[pcvl.BasicState([np.sign(i) for i in k])] += v
        except:
            pcvl_td[pcvl.BasicState([np.sign(i) for i in k])] = v

    tvd = 0
    if method == 'PNR':
        max_pht = max(size if truncation is None else truncation, max(k.n for k in pcvl_dist.keys()))
        for i in product(*[list(range(0, max_pht))]*size):
            tvd += abs(zpg_dist[i] - pcvl_dist.get(pcvl.BasicState(i), 0))/2
    else:
        for i in product(*[[0, 1]]*size):
            tvd += abs(zpg_dist[i] - pcvl_td.get(pcvl.BasicState(i), 0))/2

    return {'time': tf, 'tvd': tvd}

In [11]:
size = 3
par = {'area': np.pi, 'width': 0.001}  # TVD -> 0 as width -> 0, indicating accurate simulation
U = qt.rand_unitary_haar(size)
print(tvd(U, tlist=[0, 30], method='Threshold', args=par))
print(tvd(U, tlist=[0, 30], method='PNR', truncation=size+1, args=par))

{'time': 0.07523894309997559, 'tvd': 0.00011184133669126742}
{'time': 0.720811128616333, 'tvd': 0.0003708077665223209}
